In [ ]:
from grid.interface import *

In [ ]:
from boardlaw.backup import *

In [ ]:
sync_up('./output/pavlov', 'boardlaw:output/pavlov')
sync_up('./output/experiments/bee', 'boardlaw:output/experiments/bee')

In [ ]:
from grid.eval import *
vits = pd.concat([
    vitals(3),
    vitals(4),
    vitals(5),
    vitals(7),
    vitals(9)], 0)

In [ ]:
v = vitals(6)

In [ ]:
vits = v.combine_first(vits)

In [ ]:
(pn.ggplot(data=vits.query('boardsize in (7, 9)'))
    + pn.geom_text(pn.aes(x='flops', y='μ', label='width'))
    + pn.scale_x_continous(tran))

In [ ]:
boardsize = 4

In [ ]:
snaps = snapshots(boardsize)
snaps = pd.concat([snaps, parameters(snaps)], 1)
snaps['nickname'] = snaps.run.str.extract('.* (.*)', expand=False) + '.' + snaps.idx.astype(str)
snaps['params'] = params(snaps)
snaps = snaps.set_index('nickname')

In [ ]:
games, wins = load(boardsize, snaps.index)

In [ ]:
soln = activelo.solve(games, wins)

In [ ]:
snaps['μ'], snaps['σ'] = arena.analysis.difference(soln, soln.μ.idxmax())

In [ ]:
plot_flops(snaps)

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(games)